In [8]:
# Pre-Process CMV Data
# ====================

import pandas as pd
import json
import re
import os
from IPython.display import Markdown
import random

In [9]:
print(os.getcwd())

/Users/joshua.sheppard/PycharmProjects/countaBot/src/EDA


In [32]:
conan = [json.loads(ln) for ln in open("../data/raw/CONAN.json", "r")]
conan_multi = [json.loads(ln) for ln in open("../data/raw/multitarget_CONAN.json", "r")]

print(len(conan[0]["conan"]))
print(len(conan_multi[0]))

conan
conan_multi

14988
5003


[{'0': {'HATE_SPEECH': "Maybe the UN could talk to those asian and african nations responsible for 90%+ of the pollution in the oceans' instead of insisting on this bullshit about climate change.",
   'COUNTER_NARRATIVE': "You're right, ocean pollution is one of the main environmental problems we face, however, climate change is a global problem, and everyone should help to combat it, no matter where in the world you are from!",
   'TARGET': 'other',
   'VERSION': 'V1'},
  '1': {'HATE_SPEECH': '‘Juice (Jews) are worse that nukes.’ Caption to a meme showing Hiroshima 1945 and present day, contrast against pictures of Detroit in 1945 and present day.’',
   'COUNTER_NARRATIVE': "You'd rather be nuked then?",
   'TARGET': 'JEWS',
   'VERSION': 'V1'},
  '2': {'HATE_SPEECH': 'Foreigners on UK benefits leaps 41% in 5 years with over 400K now handed payouts. This nonsense has got to stop.',
   'COUNTER_NARRATIVE': 'You seem to be worried about government expenditure… it has actually been shown

In [29]:
ids = []
hate = []
counter = []
type = []
sub_type = []

for _ in conan[0]["conan"]:
    if _["cn_id"][0:3] == "ENT":
        ids.append(_["cn_id"])
        hate.append(_["hateSpeech"])
        counter.append(_["counterSpeech"])
        type.append(_["hsType"])
        sub_type.append(_["hsSubType"])

print(len(hate))

3864


In [33]:
conan[0]

{'conan': [{'cn_id': 'ENT1ST0001HS0033CN000021',
   'hateSpeech': 'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.',
   'counterSpeech': 'To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.',
   'hsType': 'Islamophobia',
   'hsSubType': 'crimes',
   'cnType': 'facts',
   'age': 61.0,
   'gender': 'male',
   'educationLevel': 'Bachelor'},
  {'cn_id': 'ENT1ST0001HS0033CN000021P1',
   'hateSpeech': "In Birmingham there is a school where girls and boys are separeted even if it isn't legal.",
   'counterSpeech': 'To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.',
   'hsType': 'Islamophobia',
   'hsSubType': 'crimes',
   'cnType': 'facts',
   'age': 61.0,
   'gender': 'male',
   'educationLevel': 'Bachelor'},
 

In [31]:
sub_type
counter
type
hate

['According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.',
 "In Birmingham there is a school where girls and boys are separeted even if it isn't legal.",
 'A school in Birningham is still separating girls from boys. This information come from a Ofsted report. A high court said in 2017 that this type of action is illegal.',
 'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.',
 "In Birmingham there is a school where girls and boys are separeted even if it isn't legal.",
 'A school in Birningham is still separating girls from boys. This information come from a Ofsted report. A high court said in 2017 that this type of action is illegal.',
 'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this 

In [15]:
multi_id_ = []
multi_hate = []
multi_counter = []

for _ in range(0, len(conan_multi[0])):
    multi_id_.append(_)
    multi_hate.append(conan_multi[0][str(_)]["HATE_SPEECH"])
    multi_counter.append(conan_multi[0][str(_)]["COUNTER_NARRATIVE"])

print(len(multi_hate))

5003


In [20]:
# Clean Posy

# Display Post IPython Markdown
def show_post(hate, counter, type):
    # cmv_post = cleanup(cmv_post)
    md_format = "**{}** \n \n {} \n \n **Counter** \n \n {}".format(hate, counter, type)
    md_format = "\n".join(["> " + line for line in md_format.splitlines()])

    return Markdown(md_format)

# Clean Text
def full_clean(data):
    cleaned = []
    for i in data:
        clean = i.lower()
        # RegEx clean-up
        clean = re.sub("CMV:", " ", clean)
        clean = re.sub("CMV", " ", clean)
        clean = re.sub("cmv", " ", clean)
        clean = re.sub(r'(\.)(?:[A-Z])',r'\1\n', clean)
        clean = re.sub(r"http\S+", "", clean)
        clean = re.sub(r"\n", "", clean)
        clean = re.sub(r'(?<=[a-z])\'(?=[a-z])', '', clean)
        clean = re.sub('([^a-zA-Z\s.!?])', "", clean)
        clean = re.sub('\s+', ' ', clean)
        clean = re.sub(r"www\S+", "", clean)

        cleaned.append(clean.strip())

    return cleaned

In [21]:
len(ids), len(hate), len(full_clean(hate))

(3864, 3864, 3864)

In [22]:
### CONSTRUCT DATAFRAME OBJECTS ###

hate_clean = full_clean(hate)
counter_clean = full_clean(counter)
type_clean = full_clean(type)
sub_type_clean = full_clean(sub_type)

hate_obj = {
    "id": ids,
    "hate": hate_clean,
    "counter": counter_clean,
    "type": type_clean,
    "sub_type": sub_type_clean,
}

hate_df = pd.DataFrame(hate_obj).astype(str)

In [24]:
# Explore Post
_ = random.randint(0, len(hate_df))
show_post(hate_df["id"][_], hate_df["hate"][_], hate_df["counter"][_])

> **ENT1ST0015HS0114CN000917P2** 
>  
>  our government forgot about our christianity? for long time we fought muslims putting an end to their brutality. and now we support them? how could we do such a thing? soldiers must solve this problem because this is the real conflict. 
>  
>  **Counter** 
>  
>  what you said is an extreme case. one of the islamic principle is to respect for human dignity. islam condemns violence.

In [25]:
# Exploritory Search Keywords; Assert Clean (URLs: 'http', 'www')
keyword = "www"
hate_df[hate_df['counter'].str.contains(keyword,case=False)]

,id,hate,counter,type,sub_type


In [26]:
# Search Keywords; Assert missing values at start of sentence (known argument issue)
keyword = "harassment"
hate_df[hate_df['counter'].str.contains(keyword,case=False)]

,id,hate,counter,type,sub_type
3141,ENT1ST0016HS0094CN001048,our government has brought islamic invaders to...,and what do girls and women have to say about ...,islamophobia,islamization rapism
3142,ENT1ST0016HS0094CN001048P1,women and girls are harmed by islamic that wer...,and what do girls and women have to say about ...,islamophobia,islamization rapism
3143,ENT1ST0016HS0094CN001048P2,our women and girls are frequently victim of r...,and what do girls and women have to say about ...,islamophobia,islamization rapism


In [28]:
# Output JSON List .jsonl
import json

hate_data = []
for idx, row in hate_df.iterrows():
    hate_data.append({
        "id": row["id"],
        "hate": row["hate"],
        "counter": row["counter"],
        "type": row["type"],
        "sub_type": row["sub_type"]
    })

with open("../data/processed/hate_cleaned.jsonl", "w", encoding='utf-8') as f:
    for d in hate_data:
        f.write(json.dumps(d))
        f.write("\n")